# Utilitários Unificados — Modelos, Datasets e Ruído

Este arquivo reúne três funções principais, com interface unificada e documentação clara:

1. `load_model_unificado(modelo, caminho, ...)`
   - Carrega qualquer modelo: `linear`, `mlp`, `lstm` (Keras) ou `tft` (PyTorch Forecasting).
   - Aceita arquivo/diretório EXATO ou uma pasta raiz para descoberta recursiva.

2. Carregamento de dados (Parquet)
   - `linear/mlp/lstm` → Parquet com `*.meta.json` contendo `x_dim`, `y_dim` (e `seq_len`, `lead` no LSTM).
   - `tft` → Parquet (padrão: retorna DataFrame; opcional: cria `TimeSeriesDataSet`).

3. `add_noise_features(obj, sigma, tipo, ...)`
   - Adiciona ruído GAUSSIANO somente nas FEATURES.
   - `tipo='tfdata'` → aplica em `tf.data.Dataset` (x,y).
   - `tipo='tft'` → aplica em batches de `TimeSeriesDataSet`/`DataLoader` (chaves `encoder_cont`/`decoder_cont`).

> Observação: O notebook foi simplificado para Parquet apenas (sem TFRecords).

## 1) Carregamento Unificado de Modelos

Contrato rápido:
- Entradas:
  - `modelo`: `linear` | `mlp` | `lstm` | `tft`
  - `caminho`: arquivo/diretório exato OU uma pasta para varredura recursiva
  - `prefer_exts` (opcional): lista de extensões a priorizar (ex.: `[".cpfg", ".ckpt"]` para TFT)
  - `allow_unsafe` (bool): permite desserialização insegura apenas para artefatos LOCAIS (Lambda em Keras)
- Saídas: `(obj_modelo, info)`
  - `obj_modelo`: instancia do modelo carregado (Keras ou TemporalFusionTransformer)
  - `info`: dicionário com metadados úteis (`path`, `backend`, `kind`)

In [1]:
# Carregamento dos modelos treinados
from tensorflow import keras
import json, os
import tensorflow as tf
from keras.saving import register_keras_serializable  

@register_keras_serializable()
def repeat_to_seq_len(inputs):
    emb, feats = inputs
    seq_len = tf.shape(feats)[1]
    emb_tiled = tf.tile(emb, [1, seq_len, 1])  # (batch, seq_len, emb_dim)
    return emb_tiled

def load_model(path: str):
    if not os.path.exists(path):
        raise FileNotFoundError(f"❌ File not found: {path}")

    ext = os.path.splitext(path)[1].lower()

    # === Load model ===
    if ext == ".keras":
        # ✅ repeat dynamically with tf.tile (preserves shape)
        model = keras.models.load_model(path)
        print("✅ TensorFlow model loaded.")
    else:
        raise ValueError(f"❌ Unsupported file extension: {ext}")

    # === Load optional JSON config ===
    json_path = f"{os.path.splitext(path)[0]}.model.json"
    config = None
    if os.path.exists(json_path):
        with open(json_path, "r") as f:
            config = json.load(f)
        print(f"🧩 Loaded config: {json_path}")

    return model, config


## Carregando modelo linear
linear, info_linear = load_model('./modelos/treinamento/linear.keras')

## Carregando modelo MLP
mlp, info_mlp = load_model('./modelos/treinamento/mlp.keras')

## Carregando modelo LSTM
lstm, info_lstm = load_model('./modelos/treinamento/lstm.keras')

model_list = [
    (linear, info_linear),
    (mlp, info_mlp),
    (lstm, info_lstm)
]

2025-11-14 04:00:04.194918: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1763103606.304150   16775 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:06:00.0, compute capability: 8.6
/home/victor-bertini/Documentos/tcc_2025/TCC-2025/tfc_venv_keras/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


✅ TensorFlow model loaded.
✅ TensorFlow model loaded.
✅ TensorFlow model loaded.


## 2) Carregando preprocessadores

In [2]:
import glob
import pickle
import tensorflow as tf
from typing import Dict, Any, List, Tuple, Optional

# === LINEAR ===
linear_path = "./data/treinamento/preprocessor/linear_preproc.pkl"
with open(linear_path, "rb") as f:
    linear_preproc = pickle.load(f)


# === LSTM ===
lstm_path = "./data/treinamento/preprocessor/lstm_preproc.pkl"
with open(lstm_path, "rb") as f:
    lstm_preproc = pickle.load(f)

# === PREPROCESSORS DICT ===
preprocessors = {
    "linear": linear_preproc,
    "mlp": linear_preproc,  # MLP usa o mesmo pré-processador do Linear
    "lstm": lstm_preproc
}

print("✅ Todos os preprocessadores carregados com sucesso.")


✅ Todos os preprocessadores carregados com sucesso.


# Funções helper para análise dos modelos

## Funções de coleta de dados

In [3]:

import pandas as pd
from preprocessor_keras import LinearPreprocessor, LSTMPreprocessor
import numpy as np

def load_dataset_info(model_type: str, dataset_type: str, problem_name: str) -> Dict[str, Any]:
    info_path = f'./data/{problem_name}/{model_type}_dataset_{dataset_type}.meta.json'
    if not os.path.exists(info_path):
        raise FileNotFoundError(f"❌ Dataset info file not found: {info_path}")
    with open(info_path, 'r') as f:
        info = json.load(f)
    return info



def get_problem_df(model_type: str, lag, lead, country_list, problem_name) -> pd.DataFrame:
    # Instanciando preprocessadores
    dataset_info = load_dataset_info(model_type, "test", problem_name)
    destino_dir = f'./data/{problem_name}'

    if model_type == 'linear':
        df, dataset_info = LinearPreprocessor.load_linear_parquet_dataset(
        data_dir=destino_dir,
        split='test',
        batch_size=256,
        shuffle=True
        )
    elif model_type == 'lstm':
        df,dataset_info = LSTMPreprocessor.load_lstm_parquet_dataset(
        data_dir=destino_dir,
        split='test',
        batch_size=256,
        shuffle=True
        )
    else:
        raise ValueError(f"Modelo desconhecido: {model_type}")
    return df, dataset_info

## Funções de avaliação

In [4]:
import os
import shutil
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
from typing import Optional, Dict, Any

def avaliar_modelo_keras(
    model,
    dataset,
    titulo="Avaliação do Modelo",
    problem_name="problema",
    max_samples=None,
    n_leads=1,
    save_dir="./resultados/graficos",
    show_plots=True,
    df_info={},
    preproc=None
):
    print(f"🚀 Avaliando modelo '{model.name}'...")

    # ==========================================================
    # 1) EVALUATE
    # ==========================================================
    try:
        eval_out = model.evaluate(dataset, verbose=0, return_dict=True)
        resultados = {k: float(v) for k, v in eval_out.items()}
    except Exception:
        try:
            raw = model.evaluate(dataset, verbose=0)
            raw = raw if isinstance(raw, (list, tuple)) else [raw]
            names = model.metrics_names or [f"metric_{i}" for i in range(len(raw))]
            resultados = {n: float(v) for n, v in zip(names, raw)}
        except:
            resultados = {}

    # ==========================================================
    # 2) PREDICT
    # ==========================================================
    # Detect model type once
    is_linear = "linear" in df_info["basename"]
    is_lstm   = "lstm"   in df_info["basename"]

    X_parts = []
    preds = []
    trues = []

    for Xb, Yb in dataset:
        preds.append(model.predict(Xb, verbose=0))
        trues.append(Yb)

        if is_linear:
            # Xb is already a tensor (batch, features)
            X_parts.append(Xb.numpy())

        elif is_lstm:
            # LSTM/X uses dict inputs
            X_parts.append(Xb["num_feats"].numpy())

        else:
            raise ValueError(f"Unknown model basename: {df_info['basename']}")

    # === Now safe to concatenate ===
    Y_pred = np.concatenate(preds, axis=0)
    Y_real = np.concatenate(trues, axis=0)
    X_vals = np.concatenate(X_parts, axis=0)

    # ==========================================================
    # 3) LIMIT SAMPLE SIZE
    # ==========================================================
    max_samples = len(Y_real) if not max_samples or max_samples <= 0 else min(max_samples, len(Y_real))
    Y_real, Y_pred, X_vals = Y_real[:max_samples], Y_pred[:max_samples], X_vals[:max_samples]

    # ==========================================================
    # 4) CREATE DF FROM MODEL INPUT
    # ==========================================================
    df_x = pd.DataFrame()
    df_y_real = pd.DataFrame()
    df_y_pred = pd.DataFrame()

    if is_lstm:
        # Un sqe2sqe reshape
        Y_real = Y_real.squeeze(-1)
        Y_pred = Y_pred.squeeze(-1)
        X_vals = X_vals[:, -1, :]
        tmp = []
        for col in df_info["target_cols"]:
            for lead in range(1, df_info["lead"]+1):
                tmp.append(f"{col}_lead{lead}")
        df_info["target_cols"] = tmp
            
    df_x = pd.DataFrame(X_vals, columns=df_info['feature_cols'])
    df_y_real = pd.DataFrame(Y_real, columns=df_info['target_cols'])
    df_y_pred = pd.DataFrame(Y_pred, columns=df_info['target_cols'])


    # ==========================================================
    # 5) DENORMALIZE + DECODE
    # ==========================================================
    if preproc:
        df_x = preproc.denormalize('standard', df=df_x)
        df_y_pred = preproc.denormalize('standard', df=df_y_pred)
        df_y_real = preproc.denormalize('standard', df=df_y_real)

        df_x = preproc.decode('label', df=df_x, target_col="country")
        df_x = preproc.decode('time_cycle', df=df_x)

    # ==========================================================
    # 6) COMBINE EVERYTHING INTO A SINGLE DF
    # ==========================================================
    df_full = df_x.copy()

    # Attach real + predicted columns
    for col in df_y_real.columns:
        df_full[f"{col}_real"] = df_y_real[col].values
    for col in df_y_pred.columns:
        df_full[f"{col}_pred"] = df_y_pred[col].values

    # Detect country columns
    country_cols = [c for c in df_full.columns if "country" in c.lower()]

    # Sorting order
    order_cols = []
    if "datetime" in df_full.columns:
        order_cols.append("datetime")
    order_cols.extend(country_cols)

    if order_cols:
        df_full = df_full.sort_values(order_cols).reset_index(drop=True)
    else:
        print("⚠️ Nenhuma coluna para ordenar. Mantendo ordem original.")

    df_combined = df_full  # keep this for later use

    # ==========================================================
    # 7) OUTPUT DIR
    # ==========================================================
    output_dir = os.path.join(save_dir, problem_name, model.name)
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    # ==========================================================
    # 8) GLOBAL METRICS
    # ==========================================================
    yt_all = df_y_real.values.reshape(-1)
    yp_all = df_y_pred.values.reshape(-1)
    diff = yp_all - yt_all

    mae = float(np.mean(np.abs(diff)))
    mse = float(np.mean(diff ** 2))
    rmse = float(np.sqrt(mse))

    corr = float(np.corrcoef(yt_all, yp_all)[0, 1]) \
        if np.std(yt_all) > 0 and np.std(yp_all) > 0 else float("nan")

    ss_res = np.sum((yt_all - yp_all) ** 2)
    ss_tot = np.sum((yt_all - np.mean(yt_all)) ** 2)
    r2 = float(1 - ss_res / ss_tot) if ss_tot > 0 else float("nan")

    resultados.update(dict(mae=mae, mse=mse, rmse=rmse, correlacao_pearson=corr, r2=r2))

    # ==========================================================
    # 9) PER-LEAD METRICS
    # ==========================================================
    per_lead = {}
    lead_cols = df_y_real.columns

    for col in lead_cols:
        t = df_y_real[col].values
        p = df_y_pred[col].values
        d = p - t

        mae_i = float(np.mean(np.abs(d)))
        mse_i = float(np.mean(d ** 2))
        rmse_i = float(np.sqrt(mse_i))

        corr_i = float(np.corrcoef(t, p)[0, 1]) \
            if np.std(t) > 0 and np.std(p) > 0 else float("nan")

        ss_res = np.sum((t - p) ** 2)
        ss_tot = np.sum((t - np.mean(t)) ** 2)
        r2_i = float(1 - ss_res / ss_tot) if ss_tot > 0 else float("nan")

        for key, val in zip(["mae", "mse", "rmse", "correlacao_pearson", "r2"],
                            [mae_i, mse_i, rmse_i, corr_i, r2_i]):
            per_lead.setdefault(key, []).append(val)

    # ==========================================================
    # 10) SAVE OVERALL METRIC TABLE
    # ==========================================================
    num_items = [(k, v) for k, v in resultados.items() if isinstance(v, (int, float))]

    fig, ax = plt.subplots(figsize=(8, 0.4 * len(num_items) + 1))
    ax.axis("off")
    ax.table(
        cellText=[[k, f"{v:.6f}"] for k, v in num_items],
        colLabels=["Métrica", "Valor"],
        loc="center",
    )
    plt.tight_layout()
    fig.savefig(os.path.join(output_dir, "overall_metrics.png"), dpi=150)
    plt.show() if show_plots else plt.close()

    # ==========================================================
    # 11) PLOT METRIC PER LEAD
    # ==========================================================
    for metric, vals in per_lead.items():
        xs = np.arange(1, len(vals) + 1)
        plt.figure(figsize=(9, 4))
        plt.plot(xs, vals, marker="o")
        plt.title(f"{titulo} — {metric.upper()} por Lead")
        plt.xlabel("Lead")
        plt.ylabel(metric.upper())
        plt.grid(True, linestyle="--", alpha=0.6)
        plt.tight_layout()
        fp = os.path.join(output_dir, f"metric_{metric}.png")
        plt.savefig(fp, dpi=150)
        plt.show() if show_plots else plt.close()

    # ==========================================================
    # 12) PLOTS POR PAÍS E LEAD
    # ==========================================================
    datetime_col = "datetime"
    if datetime_col not in df_combined.columns:
        raise ValueError("Coluna 'datetime' não encontrada em df_combined.")

    if len(country_cols) >= 1:
        main_country_col = country_cols[0]
        unique_countries = df_combined[main_country_col].unique()
    else:
        unique_countries = [None]

    for country in unique_countries:
        if country is not None:
            df_plot = df_combined[df_combined[main_country_col] == country]
            tag = f"{main_country_col}_{country}"
        else:
            df_plot = df_combined
            tag = "all"

        time_axis = pd.to_datetime(df_plot[datetime_col])

        for col in lead_cols:
            real_col = f"{col}_real"
            pred_col = f"{col}_pred"

            plt.figure(figsize=(12, 5))
            plt.plot(time_axis, df_plot[real_col], label=f"Real ({col})", linewidth=2)
            plt.plot(time_axis, df_plot[pred_col],
                     label=f"Pred ({col})", linestyle="--", linewidth=2)

            plt.title(f"{titulo} — {col} — {tag}")
            plt.xlabel("Tempo")
            plt.ylabel("Valor")
            plt.grid(True, linestyle="--", alpha=0.6)
            plt.legend()

            fp = os.path.join(output_dir, f"{tag}_lead_{col}.png")
            plt.tight_layout()
            plt.savefig(fp, dpi=150)
            plt.show() if show_plots else plt.close()

    print("✅ Avaliação concluída.")

    return dict(
        resultados,
        y_true=Y_real,
        y_pred=Y_pred,
        per_lead_metrics=per_lead,
        df=df_combined
    )


# N1A — Série Univariada (seq_len=72, lead=72)


Objetivo
- Prever 24 horas de carga à frente com janelas de 48 horas de histórico para um único país.


Artefatos esperados
- Parquet (Linear/MLP): `data/N1A/linear_dataset_{split}.parquet` + `linear_dataset_{split}.meta.json` → { x_dim, y_dim }
- Parquet (LSTM): `data/N1A/lstm_dataset_{split}.parquet` + `lstm_dataset_{split}.meta.json` → { seq_len=240, lead=72, x_dim, y_dim }


Modelos a comparar
- Linear, MLP, LSTM (Keras) e, opcionalmente, TFT.


Métricas e checks
- MAE, RMSE, MAPE.
- Checar: shapes conforme meta.json; ausência de NaNs; número de amostras > 0.


Visualizações sugeridas
- Boxplot de erro por horizonte; barras de MAE por modelo; curva MAE vs horizonte.


Notas
- As variantes A/B são obtidas reduzindo a janela/horizonte efetivos na avaliação a partir do dataset base (240/72).
- Padding (se houver) deve usar sentinela fixo para permitir mascaramento e ruído seletivo.

In [5]:
# Carregamento dos dataset N1A (Parquet)

# linear/mlp (iguais)
ds_linear, linear_info = get_problem_df(
    model_type='linear',
    lag=72,
    lead=72,
    country_list=['ES'],
    problem_name='N1A'
)

# lstm
ds_lstm, lstm_info = get_problem_df(
    model_type='lstm',
    lag=72,
    lead=72,
    country_list=['ES'],
    problem_name='N1A'
)


## Avaliando dados

### Avaliando modelo Linear
avaliar_modelo_keras(
    model=linear,
    dataset=ds_linear,
    titulo="Modelo Linear - Problema N1A",
    problem_name="N1A",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['linear'],   
    df_info=linear_info
)

### Avaliando modelo MLP
avaliar_modelo_keras(
    model=mlp,
    dataset=ds_linear,
    titulo="Modelo MLP - Problema N1A",
    problem_name="N1A",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['mlp'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo LSTM
avaliar_modelo_keras(
    model=lstm,
    dataset=ds_lstm,
    titulo="Modelo lstm - Problema N1A",
    problem_name="N1A",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['lstm'],
               
            
                
    df_info=lstm_info
)

🚀 Avaliando modelo 'linear_model'...


2025-11-14 04:00:09.013538: I external/local_xla/xla/service/service.cc:163] XLA service 0x2c9d3f10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-14 04:00:09.013550: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-11-14 04:00:09.033406: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-14 04:00:09.066616: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91501
I0000 00:00:1763103609.572524   16839 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-11-14 04:00:09.595928: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints s

✅ Avaliação concluída.
🚀 Avaliando modelo 'mlp_model'...


2025-11-14 04:00:28.310913: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-14 04:00:28.310941: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-14 04:00:29.353911: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_59', 116 bytes spill stores, 116 bytes spill loads

2025-11-14 04:00:29.693672: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were fi

✅ Avaliação concluída.
🚀 Avaliando modelo 'lstm'...


/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de

✅ Avaliação concluída.


{'loss': 0.017111890017986298,
 'mae': 1973.6455078125,
 'mse': 6395437.0,
 'rmse': 2528.9201252708635,
 'correlacao_pearson': 0.7200422972444838,
 'r2': 0.5145313739776611,
 'y_true': array([[ 0.19495066,  0.09108222, -0.06492734, ...,  0.034596  ,
          0.07804694,  0.14405099],
        [-0.01071712, -0.00430293, -0.01051021, ..., -0.08479062,
          0.09211677,  0.12460151],
        [-0.45350286, -0.40881047, -0.38584355, ..., -0.32728824,
         -0.32004642, -0.27225038],
        ...,
        [-0.06927243, -0.02664913,  0.02528508, ...,  0.02507818,
          0.00955998, -0.01361385],
        [ 0.11611824,  0.18791574,  0.09087531, ..., -0.22031617,
         -0.16755432, -0.12824155],
        [-0.01775203,  0.02156072,  0.06728765, ...,  0.04618292,
          0.01390508, -0.02168331]], shape=(1603, 72), dtype=float32),
 'y_pred': array([[ 0.08076284,  0.05755486,  0.01478296, ..., -0.02001864,
          0.0300623 ,  0.04182211],
        [-0.06729016,  0.01303364,  0.004815

# N1B — Série Univariada (seq_len=168, lead=48)


Objetivo
- Prever 48 horas de carga à frente com janelas de 168 horas de histórico para um único país.


Artefatos esperados
- Parquet (Linear/MLP): `data/N1B/linear_dataset_{split}.parquet` + meta { x_dim, y_dim }
- Parquet (LSTM): `data/N1B/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (opcional): `data/treinamento/tft_dataset_{split}.parquet`


Modelos a comparar
- Linear, MLP, LSTM, TFT (opcional).


Métricas e checks
- MAE, RMSE, MAPE; validação de shapes e ausência de NaNs.


Visualizações sugeridas
- Barras de MAE médio por modelo; curva de erro por horizonte.


Notas
- As variantes A/B são derivadas do dataset base (240/72) reduzindo janela/horizonte na avaliação, sem retreinar.

In [6]:
# Carregamento dos dataset N1B (Parquet)

# linear/mlp (iguais)
ds_linear, linear_info = get_problem_df(
    model_type='linear',
    lag=168,
    lead=72,
    country_list=['ES'],
    problem_name='N1B'
)

# lstm
ds_lstm, lstm_info = get_problem_df(
    model_type='lstm',
    lag=168,
    lead=72,
    country_list=['ES'],
    problem_name='N1B'
)


## Avaliando dados

## Avaliando modelo Linear
avaliar_modelo_keras(
    model=linear,
    dataset=ds_linear,
    titulo="Modelo Linear - Problema N1B",
    problem_name="N1B",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['linear'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo MLP
avaliar_modelo_keras(
    model=mlp,
    dataset=ds_linear,
    titulo="Modelo MLP - Problema N1B",
    problem_name="N1B",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['mlp'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo LSTM
avaliar_modelo_keras(
    model=lstm,
    dataset=ds_lstm,
    titulo="Modelo lstm - Problema N1B",
    problem_name="N1B",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['lstm'],
               
            
                
    df_info=lstm_info
)

🚀 Avaliando modelo 'linear_model'...


2025-11-14 04:01:08.697753: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/home/victor-bertini/Documentos/tcc_2025/TCC-2025/preprocessor.py:302: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tgt] = dt
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resul

✅ Avaliação concluída.
🚀 Avaliando modelo 'mlp_model'...


2025-11-14 04:01:25.656808: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-14 04:01:25.656830: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-14 04:01:25.863051: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_48', 4 bytes spill stores, 4 bytes spill loads

2025-11-14 04:01:26.369629: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Reg

✅ Avaliação concluída.
🚀 Avaliando modelo 'lstm'...


/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de

✅ Avaliação concluída.


{'loss': 0.017629796639084816,
 'mae': 2006.7801513671875,
 'mse': 6588917.0,
 'rmse': 2566.8885834800076,
 'correlacao_pearson': 0.7033125328183529,
 'r2': 0.4904373288154602,
 'y_true': array([[-0.19445251, -0.17293395, -0.1224481 , ..., -0.12224119,
          0.03749273,  0.07390875],
        [-0.33908206, -0.33432314, -0.2970795 , ..., -0.36535954,
         -0.42225957, -0.45308906],
        [-0.2482489 , -0.19010742, -0.15534668, ..., -0.00430293,
         -0.01320003, -0.0192004 ],
        ...,
        [-0.10941282, -0.0552026 , -0.11230955, ...,  0.02156072,
          0.06728765,  0.10328986],
        [ 0.11198005,  0.02466436, -0.11375792, ..., -0.02664913,
          0.02528508,  0.13287789],
        [-0.16031249, -0.20355652, -0.19445251, ..., -0.3353577 ,
         -0.2813544 , -0.12224119]], shape=(1507, 72), dtype=float32),
 'y_pred': array([[-0.03841551, -0.02354192,  0.02884105, ..., -0.02298776,
         -0.06705464, -0.04423872],
        [-0.35073158, -0.30959976, -0.285

# N1C — Série Univariada (seq_len=240, lead=72)


Objetivo
- Pré-treino/treino com a janela de 240 horas e avaliar horizonte de 72 horas.


Artefatos esperados
- Parquet (Linear/MLP): `data/N1C/linear_dataset_{split}.parquet` + meta { x_dim, y_dim }
- Parquet (LSTM): `data/N1C/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (opcional): `data/treinamento/tft_dataset_{split}.parquet`


Modelos a comparar
- Linear, MLP, LSTM, TFT (opcional).


Métricas e checks
- MAE, RMSE, MAPE; número de amostras por split; coerência entre seq_len/lead do meta e shapes efetivos.


Visualizações sugeridas
- Curva comparativa de MAE vs horizonte; top‑k modelos por MAE.


Notas
- Esta variante (C) é a base máxima de lookback e horizonte; A/B são obtidas por redução na avaliação.

In [7]:
# Carregamento dos dataset N1C (Parquet)

# linear/mlp (iguais)
ds_linear, linear_info = get_problem_df(
    model_type='linear',
    lag=240,
    lead=72,
    country_list=['ES'],
    problem_name='N1C'
)

# lstm
ds_lstm, lstm_info = get_problem_df(
    model_type='lstm',
    lag=240,
    lead=72,
    country_list=['ES'],
    problem_name='N1C'
)


## Avaliando dados

## Avaliando modelo Linear
avaliar_modelo_keras(
    model=linear,
    dataset=ds_linear,
    titulo="Modelo Linear - Problema N1C",
    problem_name="N1C",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['linear'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo MLP
avaliar_modelo_keras(
    model=mlp,
    dataset=ds_linear,
    titulo="Modelo MLP - Problema N1C",
    problem_name="N1C",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['mlp'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo LSTM
avaliar_modelo_keras(
    model=lstm,
    dataset=ds_lstm,
    titulo="Modelo lstm - Problema N1C",
    problem_name="N1C",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['lstm'],
               
            
                
    df_info=lstm_info
)

🚀 Avaliando modelo 'linear_model'...


2025-11-14 04:02:01.265741: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-14 04:02:01.902506: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_8', 164 bytes spill stores, 164 bytes spill loads

/home/victor-bertini/Documentos/tcc_2025/TCC-2025/preprocessor.py:302: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tgt] = dt
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highl

✅ Avaliação concluída.
🚀 Avaliando modelo 'mlp_model'...


2025-11-14 04:02:19.194787: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-14 04:02:19.194809: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-14 04:02:19.388304: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_48', 4 bytes spill stores, 4 bytes spill loads

2025-11-14 04:02:19.572005: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Reg

✅ Avaliação concluída.
🚀 Avaliando modelo 'lstm'...


/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de

✅ Avaliação concluída.


{'loss': 0.016942283138632774,
 'mae': 1965.6788330078125,
 'mse': 6331794.0,
 'rmse': 2516.305625316607,
 'correlacao_pearson': 0.7196563090535026,
 'r2': 0.5174899101257324,
 'y_true': array([[-0.22362672, -0.2232129 , -0.1739685 , ..., -0.04154659,
         -0.11106809, -0.1472772 ],
        [-0.33059877, -0.31280458, -0.2703882 , ..., -0.20376343,
         -0.2656293 , -0.30928713],
        [ 0.05735601,  0.13205025,  0.0592182 , ..., -0.06451352,
         -0.01382076,  0.02900945],
        ...,
        [ 0.21377939,  0.21729684,  0.24667795, ..., -0.1696234 ,
         -0.12824155, -0.07403135],
        [ 0.08694404,  0.19495066,  0.24854013, ..., -0.25880128,
         -0.2482489 , -0.19010742],
        [ 0.02052617, -0.0821008 , -0.13755247, ...,  0.28888744,
          0.36337477,  0.26281688]], shape=(1435, 72), dtype=float32),
 'y_pred': array([[-0.26953763, -0.23504788, -0.21402904, ..., -0.2207626 ,
         -0.2932798 , -0.3057364 ],
        [-0.30593503, -0.30048847, -0.2938

# N2A — Múltiplos Países (seq_len=72, lead=24)


Objetivo
- Prever 24 horas com 72 horas de histórico, agrupando por país.


Artefatos esperados
- Parquet (Linear/MLP): `data/N2A/linear_dataset_{split}.parquet` + meta
- Parquet (LSTM): `data/N2A/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (recomendado): `data/treinamento/tft_dataset_{split}.parquet` (colunas: _group_id=country, time_idx crescente por grupo, quantity_MW)


Modelos a comparar
- Linear, MLP, LSTM (podem exigir codificação/flatten por grupo);
- TFT (nativamente multi‑grupo).


Métricas e checks
- MAE/RMSE por país e globais; número de grupos; equilíbrio de amostras por grupo.


Visualizações sugeridas
- Barras de MAE por modelo; facetas por país; curva MAE vs horizonte.


Notas
- As variantes A/B/C partem do dataset base (240/72), aplicando janelas/horizontes reduzidos na avaliação.

In [8]:
# Carregamento dos dataset N2A (Parquet)

# linear/mlp (iguais)
ds_linear, linear_info = get_problem_df(
    model_type='linear',
    lag=72,
    lead=72,
    country_list=['ES'],
    problem_name='N2A'
)

# lstm
ds_lstm, lstm_info = get_problem_df(
    model_type='lstm',
    lag=72,
    lead=72,
    country_list=['ES'],
    problem_name='N2A'
)


## Avaliando dados

## Avaliando modelo Linear
avaliar_modelo_keras(
    model=linear,
    dataset=ds_linear,
    titulo="Modelo Linear - Problema N2A",
    problem_name="N2A",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['linear'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo MLP
avaliar_modelo_keras(
    model=mlp,
    dataset=ds_linear,
    titulo="Modelo MLP - Problema N2A",
    problem_name="N2A",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['mlp'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo LSTM
avaliar_modelo_keras(
    model=lstm,
    dataset=ds_lstm,
    titulo="Modelo lstm - Problema N2A",
    problem_name="N2A",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['lstm'],
               
            
                
    df_info=lstm_info
)

🚀 Avaliando modelo 'linear_model'...


/home/victor-bertini/Documentos/tcc_2025/TCC-2025/preprocessor.py:302: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tgt] = dt
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fr

✅ Avaliação concluída.
🚀 Avaliando modelo 'mlp_model'...


2025-11-14 04:03:42.052209: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-14 04:03:42.052231: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-14 04:03:42.408827: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_48', 16 bytes spill stores, 16 bytes spill loads

2025-11-14 04:03:42.611741: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : R

✅ Avaliação concluída.
🚀 Avaliando modelo 'lstm'...


/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de

✅ Avaliação concluída.


{'loss': 0.01483137533068657,
 'mae': 1652.873046875,
 'mse': 5542762.5,
 'rmse': 2354.3072229426643,
 'correlacao_pearson': 0.9895384258191932,
 'r2': 0.9789261221885681,
 'y_true': array([[ 0.25919595,  0.30781963,  0.45700118, ...,  0.5700253 ,
          0.5382131 ,  0.44365555],
        [ 0.02756109, -0.04154659, -0.11106809, ..., -0.00533748,
         -0.04423641, -0.13879393],
        [ 1.2523085 ,  1.079022  ,  1.0005517 , ...,  0.68610144,
          0.7442429 ,  0.86642283],
        ...,
        [ 1.0709008 ,  0.9577732 ,  0.8758889 , ...,  0.9791883 ,
          1.0045865 ,  1.0407956 ],
        [ 0.13742988,  0.00128361, -0.12720701, ...,  0.19867502,
          0.24171214,  0.32116526],
        [-1.1697191 , -1.1461833 , -1.1051635 , ..., -1.1703916 ,
         -1.1772196 , -1.1817716 ]], shape=(4811, 72), dtype=float32),
 'y_pred': array([[ 0.30460194,  0.31036317,  0.39653438, ...,  0.70754796,
          0.5801829 ,  0.46471772],
        [ 0.04098754, -0.0058014 , -0.10155576

# N2B — Múltiplos Países (seq_len=168, lead=48)


Objetivo
- Prever 48 horas com 168 horas de histórico, agrupado por país.


Artefatos esperados
- Parquet (Linear/MLP): `data/N2B/linear_dataset_{split}.parquet` + meta
- Parquet (LSTM): `data/N2B/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (recomendado): `data/treinamento/tft_dataset_{split}.parquet` com `_group_id`, `time_idx`, target.


Modelos a comparar
- Linear, MLP, LSTM; TFT.


Métricas e checks
- MAE/RMSE por país e agregadas; distribuição de amostras por grupo.


Visualizações sugeridas
- Barras de MAE por modelo; linhas por horizonte; painel por país.


Notas
- Variantes A/B/C usam janelas/horizontes efetivos na avaliação; base: 240/72.

In [9]:
# Carregamento dos dataset N2B (Parquet)

# linear/mlp (iguais)
ds_linear, linear_info = get_problem_df(
    model_type='linear',
    lag=168,
    lead=72,
    country_list=['ES'],
    problem_name='N2B'
)

# lstm
ds_lstm, lstm_info = get_problem_df(
    model_type='lstm',
    lag=168,
    lead=72,
    country_list=['ES'],
    problem_name='N2B'
)


## Avaliando dados

## Avaliando modelo Linear
avaliar_modelo_keras(
    model=linear,
    dataset=ds_linear,
    titulo="Modelo Linear - Problema N2B",
    problem_name="N2B",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['linear'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo MLP
avaliar_modelo_keras(
    model=mlp,
    dataset=ds_linear,
    titulo="Modelo MLP - Problema N2B",
    problem_name="N2B",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['mlp'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo LSTM
avaliar_modelo_keras(
    model=lstm,
    dataset=ds_lstm,
    titulo="Modelo lstm - Problema N2B",
    problem_name="N2B",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['lstm'],
               
            
                
    df_info=lstm_info
)

🚀 Avaliando modelo 'linear_model'...


/home/victor-bertini/Documentos/tcc_2025/TCC-2025/tfc_venv_keras/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:1141: RuntimeWarning: overflow encountered in add
  X += self.mean_
/home/victor-bertini/Documentos/tcc_2025/TCC-2025/preprocessor.py:302: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tgt] = dt
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highl

✅ Avaliação concluída.
🚀 Avaliando modelo 'mlp_model'...


2025-11-14 04:06:12.142230: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-14 04:06:12.142256: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-14 04:06:12.339568: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_48', 4 bytes spill stores, 4 bytes spill loads

2025-11-14 04:06:12.832474: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Reg

✅ Avaliação concluída.
🚀 Avaliando modelo 'lstm'...


/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de

✅ Avaliação concluída.


{'loss': 0.014990217052400112,
 'mae': 1661.5799560546875,
 'mse': 5602072.0,
 'rmse': 2366.86966265572,
 'correlacao_pearson': 0.9894671313641007,
 'r2': 0.978833794593811,
 'y_true': array([[-1.0967836 , -1.1150951 , -1.1344929 , ..., -1.0615573 ,
         -1.0516257 , -1.0202789 ],
        [ 0.1107386 ,  0.12998115,  0.16660409, ...,  0.09356513,
          0.06852911,  0.0772193 ],
        [-0.09927427, -0.08975644, -0.05809933, ...,  0.17260446,
          0.18088081,  0.2048823 ],
        ...,
        [ 0.04825201,  0.04287237,  0.03314763, ..., -0.14520812,
          0.01018071,  0.04721746],
        [-1.071489  , -1.0754203 , -1.066937  , ..., -1.1633567 ,
         -1.1333032 , -1.0988528 ],
        [-1.073765  , -1.0792998 , -1.0725236 , ..., -1.0947663 ,
         -1.1071291 , -1.1131295 ]], shape=(4523, 72), dtype=float32),
 'y_pred': array([[-1.0783976 , -1.0660818 , -1.06684   , ..., -1.0202811 ,
         -1.0404934 , -1.0193295 ],
        [ 0.12626381,  0.12729327,  0.184020

# N2C — Múltiplos Países (seq_len=240, lead=72)


Objetivo
- Prever 72 horas com 240 horas de histórico, agrupado por país. Esta variante é a base para reuso em A/B.


Artefatos esperados
- Parquet (Linear/MLP): `data/N2C/linear_dataset_{split}.parquet` + meta
- Parquet (LSTM): `data/N2C/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (recomendado): `data/treinamento/tft_dataset_{split}.parquet` (agrupado por `_group_id`).


Modelos a comparar
- Linear, MLP, LSTM, TFT.


Métricas e checks
- MAE, RMSE, MAPE; comparação por país; checagem de time_idx e integridade por grupo.


Visualizações sugeridas
- Curva MAE vs horizonte; ranking de modelos por país e global.


Notas
- Base usa seq_len=240 e lead=72; variações A/B podem ser avaliadas reduzindo janela no dataset sem retreino.

In [10]:
# Carregamento dos dataset N2C (Parquet)

# linear/mlp (iguais)
ds_linear, linear_info = get_problem_df(
    model_type='linear',
    lag=240,
    lead=72,
    country_list=['ES'],
    problem_name='N2C'
)

# lstm
ds_lstm, lstm_info = get_problem_df(
    model_type='lstm',
    lag=240,
    lead=72,
    country_list=['ES'],
    problem_name='N2C'
)


## Avaliando dados

## Avaliando modelo Linear
avaliar_modelo_keras(
    model=linear,
    dataset=ds_linear,
    titulo="Modelo Linear - Problema N2C",
    problem_name="N2C",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['linear'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo MLP
avaliar_modelo_keras(
    model=mlp,
    dataset=ds_linear,
    titulo="Modelo MLP - Problema N2C",
    problem_name="N2C",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['mlp'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo LSTM
avaliar_modelo_keras(
    model=lstm,
    dataset=ds_lstm,
    titulo="Modelo lstm - Problema N2C",
    problem_name="N2C",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['lstm'],
               
            
                
    df_info=lstm_info
)

🚀 Avaliando modelo 'linear_model'...


2025-11-14 04:07:59.343047: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/home/victor-bertini/Documentos/tcc_2025/TCC-2025/preprocessor.py:302: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tgt] = dt
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resul

✅ Avaliação concluída.
🚀 Avaliando modelo 'mlp_model'...


2025-11-14 04:08:50.283593: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-14 04:08:50.283616: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-14 04:08:50.479630: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_48', 4 bytes spill stores, 4 bytes spill loads

2025-11-14 04:08:50.680521: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Reg

✅ Avaliação concluída.
🚀 Avaliando modelo 'lstm'...


/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de

✅ Avaliação concluída.


{'loss': 0.015074173919856548,
 'mae': 1664.4598388671875,
 'mse': 5633397.0,
 'rmse': 2373.4778279983993,
 'correlacao_pearson': 0.9894686312777016,
 'r2': 0.9788792133331299,
 'y_true': array([[-0.07403135, -0.12762083, -0.1874176 , ...,  0.2386085 ,
          0.12522224,  0.04018255],
        [-0.15451904,  0.01493962,  0.17156991, ..., -0.20831543,
         -0.23542054, -0.2528009 ],
        [-1.1045945 , -1.097663  , -1.0958526 , ..., -1.1411657 ,
         -1.1554941 , -1.1695123 ],
        ...,
        [-1.0357454 , -1.0617642 , -1.0871106 , ..., -1.0660577 ,
         -1.0632644 , -1.0482634 ],
        [ 1.0116214 ,  0.9770158 ,  0.8851481 , ...,  1.050779  ,
          1.1201969 ,  1.1199384 ],
        [ 0.18067391,  0.18770882,  0.2431605 , ...,  0.23178048,
          0.24605723,  0.2564027 ]], shape=(4307, 72), dtype=float32),
 'y_pred': array([[ 2.8289855e-05, -7.4819982e-02, -1.9765632e-01, ...,
          1.3819024e-02,  1.8705115e-02, -3.9810717e-02],
        [-2.1173230e-01

# N3 — Robustez a Ruído (sobre N2)

Objetivo
- Medir degradação de desempenho sob ruído gaussiano nas FEATURES (teste), mantendo rótulos intactos.

Configuração
- Conjuntos: use os datasets do N2 (A/B/C).
- Intensidades: σ ∈ {0.00, 0.01, 0.03, 0.05, 0.10}.
- Aplicação:
  - Keras/tf.data: `add_noise_features(ds, sigma, tipo='tfdata', pad_sentinel=-999.0)`.
  - TFT: `add_noise_features(tft_ds ou dataloader, sigma, tipo='tft', batch_size=..., train=False)`.

Métricas e checks
- MAE/RMSE por sigma; checar preservação de sentinela (TF) e invariância de Y.

Visualizações sugeridas
- Curvas MAE vs σ por modelo; heatmap de degradação por horizonte e sigma.

Notas
- Aplique ruído após normalização das features.
- Não altere o treino; apenas avaliação/benchmark.


## N3A

In [11]:
# Carregamento dos dataset N3A (Parquet)

# linear/mlp (iguais)
ds_linear, linear_info = get_problem_df(
    model_type='linear',
    lag=72,
    lead=72,
    country_list=['ES'],
    problem_name='N3A'
)

# lstm
ds_lstm, lstm_info = get_problem_df(
    model_type='lstm',
    lag=72,
    lead=72,
    country_list=['ES'],
    problem_name='N3A'
)


## Avaliando dados

## Avaliando modelo Linear
avaliar_modelo_keras(
    model=linear,
    dataset=ds_linear,
    titulo="Modelo Linear - Problema N3A",
    problem_name="N3A",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['linear'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo MLP
avaliar_modelo_keras(
    model=mlp,
    dataset=ds_linear,
    titulo="Modelo MLP - Problema N3A",
    problem_name="N3A",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['mlp'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo LSTM
avaliar_modelo_keras(
    model=lstm,
    dataset=ds_lstm,
    titulo="Modelo lstm - Problema N3A",
    problem_name="N3A",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['lstm'],
               
            
                
    df_info=lstm_info
)

🚀 Avaliando modelo 'linear_model'...


/home/victor-bertini/Documentos/tcc_2025/TCC-2025/preprocessor.py:302: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tgt] = dt
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fr

✅ Avaliação concluída.
🚀 Avaliando modelo 'mlp_model'...


/home/victor-bertini/Documentos/tcc_2025/TCC-2025/preprocessor.py:302: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tgt] = dt
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fr

✅ Avaliação concluída.
🚀 Avaliando modelo 'lstm'...


/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de

✅ Avaliação concluída.


{'loss': 0.014822191558778286,
 'mae': 1652.322265625,
 'mse': 5539364.0,
 'rmse': 2353.585350056377,
 'correlacao_pearson': 0.9895454598009561,
 'r2': 0.9789390563964844,
 'y_true': array([[ 0.25919595,  0.30781963,  0.45700118, ...,  0.5700253 ,
          0.5382131 ,  0.44365555],
        [ 0.02756109, -0.04154659, -0.11106809, ..., -0.00533748,
         -0.04423641, -0.13879393],
        [ 1.2523085 ,  1.079022  ,  1.0005517 , ...,  0.68610144,
          0.7442429 ,  0.86642283],
        ...,
        [ 1.0709008 ,  0.9577732 ,  0.8758889 , ...,  0.9791883 ,
          1.0045865 ,  1.0407956 ],
        [ 0.13742988,  0.00128361, -0.12720701, ...,  0.19867502,
          0.24171214,  0.32116526],
        [-1.1697191 , -1.1461833 , -1.1051635 , ..., -1.1703916 ,
         -1.1772196 , -1.1817716 ]], shape=(4811, 72), dtype=float32),
 'y_pred': array([[ 0.3081988 ,  0.31417704,  0.3998248 , ...,  0.70831263,
          0.58113337,  0.46568432],
        [ 0.03937183, -0.00760787, -0.10316185

## N3B

In [12]:
# Carregamento dos dataset N3B (Parquet)

# linear/mlp (iguais)
ds_linear, linear_info = get_problem_df(
    model_type='linear',
    lag=168,
    lead=72,
    country_list=['ES'],
    problem_name='N3B'
)

# lstm
ds_lstm, lstm_info = get_problem_df(
    model_type='lstm',
    lag=168,
    lead=72,
    country_list=['ES'],
    problem_name='N3B'
)


## Avaliando dados

## Avaliando modelo Linear
avaliar_modelo_keras(
    model=linear,
    dataset=ds_linear,
    titulo="Modelo Linear - Problema N3B",
    problem_name="N3B",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['linear'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo MLP
avaliar_modelo_keras(
    model=mlp,
    dataset=ds_linear,
    titulo="Modelo MLP - Problema N3B",
    problem_name="N3B",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['mlp'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo LSTM
avaliar_modelo_keras(
    model=lstm,
    dataset=ds_lstm,
    titulo="Modelo lstm - Problema N3B",
    problem_name="N3B",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['lstm'],
               
            
                
    df_info=lstm_info
)

🚀 Avaliando modelo 'linear_model'...


/home/victor-bertini/Documentos/tcc_2025/TCC-2025/preprocessor.py:302: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tgt] = dt
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fr

✅ Avaliação concluída.
🚀 Avaliando modelo 'mlp_model'...


/home/victor-bertini/Documentos/tcc_2025/TCC-2025/preprocessor.py:302: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tgt] = dt
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fr

✅ Avaliação concluída.
🚀 Avaliando modelo 'lstm'...


/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de

✅ Avaliação concluída.


{'loss': 0.01499449647963047,
 'mae': 1661.9853515625,
 'mse': 5603665.5,
 'rmse': 2367.206264777111,
 'correlacao_pearson': 0.9894653093737796,
 'r2': 0.9788277745246887,
 'y_true': array([[-1.0967836 , -1.1150951 , -1.1344929 , ..., -1.0615573 ,
         -1.0516257 , -1.0202789 ],
        [ 0.1107386 ,  0.12998115,  0.16660409, ...,  0.09356513,
          0.06852911,  0.0772193 ],
        [-0.09927427, -0.08975644, -0.05809933, ...,  0.17260446,
          0.18088081,  0.2048823 ],
        ...,
        [ 0.04825201,  0.04287237,  0.03314763, ..., -0.14520812,
          0.01018071,  0.04721746],
        [-1.071489  , -1.0754203 , -1.066937  , ..., -1.1633567 ,
         -1.1333032 , -1.0988528 ],
        [-1.073765  , -1.0792998 , -1.0725236 , ..., -1.0947663 ,
         -1.1071291 , -1.1131295 ]], shape=(4523, 72), dtype=float32),
 'y_pred': array([[-1.0793276 , -1.0672221 , -1.0680572 , ..., -1.0209308 ,
         -1.0410625 , -1.0195658 ],
        [ 0.12657425,  0.12757768,  0.18439622

## N3C

In [13]:
# Carregamento dos dataset N3C (Parquet)

# linear/mlp (iguais)
ds_linear, linear_info = get_problem_df(
    model_type='linear',
    lag=240,
    lead=72,
    country_list=['ES'],
    problem_name='N3C'
)

# lstm
ds_lstm, lstm_info = get_problem_df(
    model_type='lstm',
    lag=240,
    lead=72,
    country_list=['ES'],
    problem_name='N3C'
)


## Avaliando dados

## Avaliando modelo Linear
avaliar_modelo_keras(
    model=linear,
    dataset=ds_linear,
    titulo="Modelo Linear - Problema N3C",
    problem_name="N3C",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['linear'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo MLP
avaliar_modelo_keras(
    model=mlp,
    dataset=ds_linear,
    titulo="Modelo MLP - Problema N3C",
    problem_name="N3C",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['mlp'],
               
            
                
    df_info=linear_info
)

### Avaliando modelo LSTM
avaliar_modelo_keras(
    model=lstm,
    dataset=ds_lstm,
    titulo="Modelo lstm - Problema N3C",
    problem_name="N3C",
    n_leads=72,
    max_samples=5000,
    show_plots=False,
    preproc=preprocessors['lstm'],
               
            
                
    df_info=lstm_info
)

🚀 Avaliando modelo 'linear_model'...


/home/victor-bertini/Documentos/tcc_2025/TCC-2025/preprocessor.py:302: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tgt] = dt
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fr

✅ Avaliação concluída.
🚀 Avaliando modelo 'mlp_model'...


/home/victor-bertini/Documentos/tcc_2025/TCC-2025/preprocessor.py:302: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tgt] = dt
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fr

✅ Avaliação concluída.
🚀 Avaliando modelo 'lstm'...


/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[f"{col}_pred"] = df_y_pred[col].values
/tmp/ipykernel_16775/3228294489.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de

✅ Avaliação concluída.


{'loss': 0.015083481557667255,
 'mae': 1664.928955078125,
 'mse': 5636862.5,
 'rmse': 2374.2077626020855,
 'correlacao_pearson': 0.9894626247123844,
 'r2': 0.9788662195205688,
 'y_true': array([[-0.07403135, -0.12762083, -0.1874176 , ...,  0.2386085 ,
          0.12522224,  0.04018255],
        [-0.15451904,  0.01493962,  0.17156991, ..., -0.20831543,
         -0.23542054, -0.2528009 ],
        [-1.1045945 , -1.097663  , -1.0958526 , ..., -1.1411657 ,
         -1.1554941 , -1.1695123 ],
        ...,
        [-1.0357454 , -1.0617642 , -1.0871106 , ..., -1.0660577 ,
         -1.0632644 , -1.0482634 ],
        [ 1.0116214 ,  0.9770158 ,  0.8851481 , ...,  1.050779  ,
          1.1201969 ,  1.1199384 ],
        [ 0.18067391,  0.18770882,  0.2431605 , ...,  0.23178048,
          0.24605723,  0.2564027 ]], shape=(4307, 72), dtype=float32),
 'y_pred': array([[-1.9953400e-04, -7.5093299e-02, -1.9800426e-01, ...,
          1.4047861e-02,  1.8869564e-02, -3.9711632e-02],
        [-2.1319723e-01,